# Traj-Pred Debugging

Utils for loading and visualization the input and output information of a given test run.

In [ ]:
! pip install shapely
! pip install geopy

import os
import torch
import random
from tqdm import tqdm
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
from graph_construction.graph_utils import plot_context

In [ ]:
def get_batch_distribution(batch):
    unique_values, counts = np.unique(batch['airport'], return_counts=True)
    # Print the results
    for value, count in zip(unique_values, counts):
        print(f"{value} occurs {count} times")
    return (unique_values, counts)

In [ ]:
def plot_batch(num_to_visualize, batch, tag, OUT_DIR):
    HEADS = 4
    context = batch['context']
    trajectories =  batch['Y']
    predictions = batch['mu'][:,:,:,:]
    scores = batch['pred_scores']
    for b in range(num_to_visualize):
        airport = batch['airport'][b]
        _, map_plot = plt.subplots() 
        patch = context[b]
        traj  = trajectories[b]
        if(airport != 'ksea'): continue
        map_plot.scatter(traj[:,1], traj[:,0], marker='.', color='r')
        pred  = predictions[b, : ,: ,:]
        pred_scores = scores[b, :]
        for h in range(HEADS):
            confidence = pred_scores[h]
            map_plot.scatter(pred[:10,h,1], pred[:10,h,0], marker='.', c = cm.winter(confidence), alpha=0.25, s=18, label = f"H : {h} K: {confidence}")
            map_plot.scatter(pred[10:,h,1], pred[10:,h,0], marker='.', color= cm.winter(confidence), alpha=0.25, s=18)
        plot_context(context = patch, ax = map_plot)
        plt.legend()
        plt.savefig(f'{OUT_DIR}/{tag}_{b}_{airport}.png',dpi=400)
        plt.show(block = False)

In [ ]:
LOG_DIR = "out_traj_aircraft"
batch_list = files = os.listdir(LOG_DIR)

In [ ]:
test_file = random.choice(batch_list)
print(f'Testing for {LOG_DIR}/{test_file}')
batch = np.load(f'{LOG_DIR}/{test_file}')
batch.files

In [ ]:
batch['pred_scores'].shape

In [ ]:
dist = get_batch_distribution(batch)